In [1]:
import os
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import random
import time

https://drive.google.com/drive/folders/1_MU7jTR0kTt8MFQYrUzWiWc6uM-BHG_9?usp=drive_link

In [2]:
def load_data(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            label = 1 if folder.lower() == 'ok' else 0  # 1 for OK, 0 for defects
            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (128, 128))  # Adjust the size as needed
                data.append(img)
                labels.append(label)
                
    data = np.array(data) / 255.0  # Normalize pixel values
    labels = np.array(labels)

    return data, labels

In [3]:
dataset_path = 'dataset'
data, labels = load_data(dataset_path)


In [4]:
print(len(data))

324


In [5]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])



In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(data, labels, epochs=10, validation_split=0.20)

#model.save('defect_detection_model.h5')

Epoch 1/10
9/9 [==============================] - 4s 295ms/step - loss: 0.4042 - accuracy: 0.8108 - val_loss: 1.0875 - val_accuracy: 0.4615
Epoch 2/10
9/9 [==============================] - 3s 278ms/step - loss: 0.3580 - accuracy: 0.8571 - val_loss: 0.4670 - val_accuracy: 0.7846
Epoch 3/10
9/9 [==============================] - 2s 252ms/step - loss: 0.2725 - accuracy: 0.8996 - val_loss: 0.3408 - val_accuracy: 0.8000
Epoch 4/10
9/9 [==============================] - 2s 247ms/step - loss: 0.3005 - accuracy: 0.8958 - val_loss: 0.5987 - val_accuracy: 0.7385
Epoch 5/10
9/9 [==============================] - 2s 238ms/step - loss: 0.2109 - accuracy: 0.9305 - val_loss: 0.6808 - val_accuracy: 0.7231
Epoch 6/10
9/9 [==============================] - 2s 245ms/step - loss: 0.1723 - accuracy: 0.9382 - val_loss: 0.8069 - val_accuracy: 0.6769
Epoch 7/10
9/9 [==============================] - 2s 250ms/step - loss: 0.1443 - accuracy: 0.9537 - val_loss: 0.4313 - val_accuracy: 0.8000
Epoch 8/10
9/9 [====

In [8]:
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))  # Assurez-vous que la taille correspond à celle utilisée lors de l'entraînement
    img = img / 255.0  # Normalisez les valeurs des pixels
    img = np.expand_dims(img, axis=0)  # Ajoutez une dimension pour représenter le lot (batch)

    return img

def predict_defect(model, image_path):
    preprocessed_img = load_and_preprocess_image(image_path)
    prediction = model.predict(preprocessed_img)

    # Interprétez la prédiction (1 signifie "OK", 0 signifie "défaut")
    if prediction[0, 0] > 0.5:
        return "OK"
    else:
        return "Défaut"



In [22]:
# Charger le modèle préalablement entraîné
#model = keras.models.load_model('defect_detection_model.h5')

# Exemple d'utilisation
image_path_to_predict = 'ok.jpg'
result = predict_defect(model, image_path_to_predict)
print(f"Prédiction pour {image_path_to_predict}: {result}")

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [110]:
resultf=""
for filename in os.listdir("testperso"):
    #print(filename)
    result = predict_defect(model, "testperso/" + filename)
    resultf= resultf + "prédiction pour"+ filename + " : "+ str(result) +"\n"
    


1/1 [==============================] - 0s 16ms/step


In [111]:
print (resultf)

prédiction pourWIN_20231010_15_02_24_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_02_40_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_00_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_09_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_11_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_18_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_24_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_29_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_56_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_58_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_04_02_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_04_05_Pro.jpg : Défaut



multi classes-----------------------------

In [70]:
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (255, 255))  # Assurez-vous que la taille correspond à celle utilisée lors de l'entraînement
    img = img / 255.0  # Normalisez les valeurs des pixels
    img = np.expand_dims(img, axis=0)  # Ajoutez une dimension pour représenter le lot (batch)

    return img

def predict_defect(model, image_path):
    preprocessed_img = load_and_preprocess_image(image_path)
    prediction = model.predict(preprocessed_img)

    # Interprétez la prédiction (1 signifie "OK", 0 signifie "défaut")
    if prediction[0, 0] > 0.5:
        return "OK"
    else:
        return "Défaut"


In [71]:
def load_data_multi_class(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            
            label = folder.lower()  # Utilisez le nom du dossier comme label
            print(label)
            for filename in os.listdir(folder_path):
                #print(filename)
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (255, 255))
                data.append(img)
                labels.append(label)

    data = np.array(data) / 255.0
    labels = np.array(labels)

    return data, labels

In [72]:
dataset_path = 'dataset'
data, labels = load_data_multi_class(dataset_path)

blobs
ok
spaghetti
under exstrosion


In [73]:
# Définir un dictionnaire de mapping catégorie -> numéro
categorie_mapping = {
    "blobs": 0,
    "cracks": 1,
    "ok": 2,
    "spaghetti": 3,
    "stringing": 4,
    "under exstrosion": 5
}

# Utiliser NumPy pour convertir les catégories en numéros
labels = np.array([categorie_mapping[c] for c in labels])

In [74]:
print (len(labels))
print(labels[-1])

177
5


In [75]:
combined_data = list(zip(data,labels))

# Mélangez la liste combinée
random.shuffle(combined_data)

# Divisez à nouveau la liste combinée en deux listes distinctes
data,labels = zip(*combined_data)

labels = np.array(labels)
data = np.array(data)
print (len(labels))
print(labels[-1])

177
2


In [76]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(255, 255, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),  # Ajout de la couche Dropout
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),  # Ajout de la couche Dropout
    layers.Dense(255, activation='relu'),
    layers.Dropout(0.5),  # Ajout de la couche Dropout
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Ajout d'une autre couche Dropout
    layers.Dense(10, activation='softmax')  # Ajustez cela en fonction de votre nombre de classes
])

In [77]:
data.shape

(177, 255, 255, 3)

In [78]:
from keras.callbacks import ModelCheckpoint

# Définissez le callback de sauvegarde du modèle
checkpoint = ModelCheckpoint('meilleur_modele.h5', monitor='val_accuracy', save_best_only=True)

# Compilez le modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraînez le modèle avec des batches de taille 10 par epoch et sauvegardez le meilleur modèle
model.fit(data, labels, epochs=10, validation_split=0.20, batch_size=10, callbacks=[checkpoint])


Epoch 1/10
15/15 [==============================] - 67s 4s/step - loss: 6.0113 - accuracy: 0.2482 - val_loss: 1.6549 - val_accuracy: 0.5278
Epoch 2/10
 5/15 [=========>....................] - ETA: 27s - loss: 1.9054 - accuracy: 0.3200

KeyboardInterrupt: 

In [ ]:
def predict_defect_multi_class(model, image_path):
    preprocessed_img = load_and_preprocess_image(image_path)
    predictions = model.predict(preprocessed_img)
    print (model.predict(preprocessed_img))

    # Interprétez les prédictions
    class_labels = ['blobs', 'cracks', 'OK', 'spaghetti','under exstrosion']
    predicted_class = class_labels[np.argmax(predictions)]

    return predicted_class

In [ ]:
resultf=""
nb_total=0
nb_positif=0
for filename in os.listdir("testperso"):
    #print(filename)
    nb_total+=1

    result = predict_defect_multi_class(model, "testperso/" + filename)
    if result=='OK':
        nb_positif+=1
        
    resultf= resultf + "prédiction pour"+ filename + " : "+ str(result) +"\n"


1/1 [==============================] - 0s 41ms/step
[[5.9661060e-04 6.4443958e-07 9.9880850e-01 4.5445759e-04 1.9350654e-07
  1.3600009e-04 3.8068251e-08 1.2353395e-07 1.1117997e-07 3.4085747e-06]]
1/1 [==============================] - 0s 48ms/step
[[9.7541037e-05 2.0789756e-13 3.9061911e-06 9.9989486e-01 1.7674442e-14
  3.6850272e-06 2.4889755e-14 8.4983764e-15 1.5818092e-17 1.3814507e-13]]
1/1 [==============================] - 0s 38ms/step
[[1.74776673e-01 7.97825919e-07 4.58881631e-03 8.20184290e-01
  5.18182510e-08 4.49096522e-04 1.66540914e-07 1.56985234e-08
  6.48843423e-10 1.00521106e-07]]
1/1 [==============================] - 0s 40ms/step
[[2.0440410e-01 3.2438166e-08 5.7968730e-01 2.1489236e-01 4.0942902e-09
  1.0162197e-03 4.7412687e-09 6.9255951e-10 4.5931497e-10 2.3398451e-08]]
1/1 [==============================] - 0s 42ms/step
[[1.1264557e-02 4.1479529e-14 9.0759257e-03 9.7963321e-01 6.7117547e-17
  2.6287304e-05 1.8223584e-15 4.0988553e-17 7.2511800e-19 7.3588689e-15]

In [ ]:
print (resultf)
print(str((nb_positif*100)/nb_total) + " % de précision")

prédiction pourok.jpg : OK
prédiction pourok3.jpg : spaghetti
prédiction pourok4.jpg : spaghetti
prédiction pourok5.png : OK
prédiction pourspaghetti.jpg : spaghetti
prédiction pourspaghetti2.jpeg : spaghetti
prédiction pourWIN_20231010_15_02_24_Pro.jpg : blobs
prédiction pourWIN_20231010_15_02_40_Pro.jpg : spaghetti
prédiction pourWIN_20231010_15_03_00_Pro.jpg : spaghetti
prédiction pourWIN_20231010_15_03_09_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_11_Pro.jpg : spaghetti
prédiction pourWIN_20231010_15_03_18_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_24_Pro.jpg : spaghetti
prédiction pourWIN_20231010_15_03_29_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_56_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_58_Pro.jpg : OK
prédiction pourWIN_20231010_15_04_02_Pro.jpg : OK
prédiction pourWIN_20231010_15_04_05_Pro.jpg : OK

50.0 % de précision


In [ ]:
# Ouvrir la webcam (la webcam par défaut a l'ID 0)
cap = cv2.VideoCapture(0)

# Vérifier si la webcam est ouverte correctement
if not cap.isOpened():
    print("Erreur: Impossible d'ouvrir la webcam.")
    exit()

# Capturer une image
time.sleep(1)
ret, frame = cap.read()

# Sauvegarder l'image capturée
if ret:
    cv2.imwrite("photo_capturee.jpg", frame)
    print("Photo capturée avec succès.")
else:
    print("Erreur lors de la capture de la photo.")

# Libérer la webcam
cap.release()
time.sleep(1)


result = predict_defect_multi_class(model, 'photo_capturee.jpg')
print(f"Prédiction pour {image_path_to_predict}: {result}")

Photo capturée avec succès.
1/1 [==============================] - 0s 40ms/step
[[0.3253603  0.02096455 0.10515112 0.32017654 0.00685089 0.19232626
  0.01053569 0.0048363  0.00487529 0.00892307]]
Prédiction pour ok.jpg: blobs
